<a href="https://colab.research.google.com/github/asanis/testcode/blob/main/DARKNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  Εισαγωγή βιβλιοθηκών
import pandas as pd
import ipaddress
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import randint

In [2]:
#  Συνάρτηση για μετατροπή IP σε αριθμούς
def ip_to_int(ip):
    try:
        return int(ipaddress.ip_address(ip))
    except ValueError:
        return None  # Επιστρέφει None αν η διεύθυνση IP είναι άκυρη

In [3]:
#  Φόρτωση δεδομένων
df = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/EKPA/refs/heads/main/Data/DarkNet.csv', low_memory=False)

In [4]:
#  Μετατροπή των IP σε αριθμούς (Src_IP & Dst_IP)
df['Src_IP'] = df['Src_IP'].apply(ip_to_int)
df['Dst_IP'] = df['Dst_IP'].apply(ip_to_int)

In [5]:
#  Διαχωρισμός χαρακτηριστικών (X) και στόχου (y)
X = df.drop(columns=['Label-1', 'Label-2'])
y = df['Label-1']

In [6]:
#  Μετατροπή όλων των δεδομένων σε αριθμούς (αντικατάσταση μη αριθμητικών τιμών)
X = X.apply(pd.to_numeric, errors='coerce')

In [7]:
#  Αντικατάσταση άπειρων τιμών με NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
#  Αντικατάσταση NaN τιμών με μέση τιμή
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [9]:
#  Κανονικοποίηση δεδομένων με StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [10]:
#  Διαχωρισμός σε σύνολο εκπαίδευσης (70%) και δοκιμής (30%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

In [11]:
#  Εξισορρόπηση των κλάσεων με SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [12]:
#  Ορισμός των υπερπαραμέτρων για βελτιστοποίηση
param_distributions = {
    'n_estimators': randint(50, 100),
    'max_depth': [10, 20, 30],
    'min_samples_split': randint(2, 5),
    'min_samples_leaf': randint(1, 2)
}


In [13]:
#  Δημιουργία Random Forest Classifier
rf = RandomForestClassifier(n_estimators=50, max_depth=15, random_state=42, min_samples_split=3, min_samples_leaf=1)

In [14]:
#  Χρήση RandomizedSearchCV για βέλτιστες υπερπαραμέτρους
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=10,
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

In [15]:
#  Εκπαίδευση του μοντέλου
random_search.fit(X_train_resampled, y_train_resampled)
best_model = random_search.best_estimator_


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [16]:
#  Αξιολόγηση του βέλτιστου μοντέλου
print("🔹 Βέλτιστες Παράμετροι:", random_search.best_params_)

🔹 Βέλτιστες Παράμετροι: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 68}


In [17]:
#  Πρόβλεψη στο σύνολο δοκιμής
y_pred = best_model.predict(X_test)

In [18]:
  #  Υπολογισμός ακρίβειας και αναφοράς ταξινόμησης
accuracy = accuracy_score(y_test, y_pred)
print(f' Ακρίβεια Μοντέλου: {accuracy}')
print('\n Αναλυτική Αξιολόγηση:')
print(classification_report(y_test, y_pred))

 Ακρίβεια Μοντέλου: 0.9963060173033926

 Αναλυτική Αξιολόγηση:
              precision    recall  f1-score   support

     Non-Tor       1.00      1.00      1.00      9279
      NonVPN       0.99      0.99      0.99      6822
         Tor       0.92      0.93      0.92       418
         VPN       1.00      1.00      1.00      4055

    accuracy                           1.00     20574
   macro avg       0.98      0.98      0.98     20574
weighted avg       1.00      1.00      1.00     20574



In [19]:
  #  Εκτύπωση του confusion matrix
print('\n Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))


 Confusion Matrix:
[[9276    2    0    1]
 [   1 6787   32    2]
 [   0   30  388    0]
 [   3    3    2 4047]]
